In [17]:
from scraper.scraper import Scraper 
bot = Scraper()
bot.accept_cookies()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/FKhan/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


In [18]:
bot.keys_search('trousers')

In [15]:
bot.next_button()

<selenium.webdriver.remote.webelement.WebElement (session="fe0138bb9509141626eff2aac1b6f60e", element="213af2bd-c837-4237-a303-6da7a99141e9")>

In [19]:
container = bot.product_container()

In [20]:
from lib2to3.pgen2 import driver
from time import time
from typing_extensions import Self
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager


In [22]:
list_properties = container.find_elements(By.XPATH, './ul/li')

In [23]:
link_list = []
for property in list_properties:
    link_list.append(property.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [40]:
import time
import uuid
property_dict = {'link':[],
                 'price':[],
                 'name':[],
                 'id' :[],
                 'UUID':[],
                 'imagelinks_1':[],
                }


for link in link_list[0:2]:
    bot.driver.get(link)
    time.sleep(1)
    property_dict['link'].append(link)
    price = bot.driver.find_element(By.XPATH, '//span[@class="pri"]')
    property_dict['price'].append(price.text)
    name = bot.driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
    property_dict['name'].append(name.text)
    id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
    property_dict['id'].append(id.text)
    property_dict['UUID'].append(uuid.uuid4())
    imageLinks_1 = bot.driver.find_element(By.TAG_NAME, 'img').get_attribute("src")
    property_dict['imagelinks_1'].append(imageLinks_1)
    

In [41]:
property_dict

{'link': ['https://www.jdsports.co.uk/product/nike-sportswear-retro-fleece-trousers/16266071/',
  'https://www.jdsports.co.uk/product/black-nike-nike-sportswear-mens-fleece-cargo-trousers/16268506/'],
 'price': ['£65.00', '£60.00'],
 'name': ['Nike Sportswear Retro Fleece Trousers',
  "Nike Nike Sportswear Men's Fleece Cargo Trousers"],
 'id': ['Product Code: 16266071/533420', 'Product Code: 16268506'],
 'UUID': [UUID('283fdeed-bc00-4294-a7b1-2cf2b489559d'),
  UUID('0f455c11-ef78-4a03-b025-d36e30995fb9')],
 'imagelinks_1': ['https://i8.amplience.net/i/jpl/globalicon-20x13-700f5f7fe1dcfead6664368748c87f4c?qlt=100',
  'https://i8.amplience.net/i/jpl/globalicon-20x13-700f5f7fe1dcfead6664368748c87f4c?qlt=100']}

In [217]:
import json
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return obj.hex
        return json.JSONEncoder.default(self, obj)
j = json.dumps(property_dict, cls=UUIDEncoder, indent=4)
f = open('data.json', 'w')
print(j, file=f)
f.close()

In [5]:
import pandas as pd 
pd.DataFrame(property_dict)

NameError: name 'property_dict' is not defined

In [6]:
import pandas as pd

In [7]:
df = pd.read_json('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike caps/data.json')
y = pd.DataFrame(df)

In [8]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dcp1.ckiqoi4xq1wx.eu-west-2.rds.amazonaws.com' 
USER = 'postgres'
PASSWORD = 'Hamburg98'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")

In [9]:
y.to_sql('Nike caps', con= engine, if_exists='append', index=False )

27

In [5]:
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [6]:
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))

In [7]:
bot.multiple_image_2(list=imageNames[0:5], file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/')

In [396]:
# this function will aid with downloading images using the specific image link (its another way of dowloading compared to the one below)
import urllib.request
def search_image():
        url = 'https://i8.amplience.net/t/jpl/jd_product_list?plu=jd_355655_plc&qlt=92&w=363&h=463&v=1&fmt=auto'
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(
        url, f'Nike_Hat_27.png')
        print(f'downloaded!')
try:
    search_image()
except: print('Thats not a valid url')

downloaded!


In [5]:
# This function saves images using the product link rather than image link and saves in chosen path
bot.image_download(url = 'https://www.jdsports.co.uk/product/adidas-originals-camo-baseball-cap/16239652/', file_name='adidas_cap_2', file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/')

In [6]:
import boto3
s3_client = boto3.client('s3')

In [47]:
# To upload multple files
import glob
png_files = glob.glob('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike caps/*.png')

for filename in png_files:
    s3_client.upload_file(filename, 'dcp1', filename)

In [32]:
# To upload individual files
response = s3_client.upload_file('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike caps/Nike_Hat_15990063_355655.png', 'dcp1', 'Nike_Hat_15990063_355655.png')

In [49]:
# To download files from a bucket
import requests
url = 'https://dcp1.s3.eu-west-2.amazonaws.com/Nike+caps/Nike_Hat_023018_023018.png'

response = requests.get(url)
with open('blastoise.png', 'wb') as f:
    f.write(response.content)